<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Improvements-pursued-in-this-notebook" data-toc-modified-id="Improvements-pursued-in-this-notebook-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Improvements pursued in this notebook</a></span></li></ul></li><li><span><a href="#Improvement-#1:-multi-label-classification" data-toc-modified-id="Improvement-#1:-multi-label-classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Improvement #1: multi-label classification</a></span><ul class="toc-item"><li><span><a href="#Performance-Tweaks" data-toc-modified-id="Performance-Tweaks-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Performance Tweaks</a></span><ul class="toc-item"><li><span><a href="#Leaky-ReLU" data-toc-modified-id="Leaky-ReLU-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Leaky ReLU</a></span></li></ul></li></ul></li></ul></div>

## Improvements pursued in this notebook

1. Change from binary classifier to multi category classifier:
    - add ims 0-9
    - add change loss fxn to cross entropy loss w/ softmax
    - change shape of final activation from 1 to 10
    - change label to 1HE
2. Add RGB

Super-short version with all of the helpers:

# Improvement #1: multi-label classification

In [ ]:
from fastai.vision.all import *

### Data ###
def init_data(path, im_size, n_cls, batch_size):
    ## Train
    # ims
    for i in range(n_cls):
        new_ims = torch.stack(
            [tensor(Image.open(fn)) for fn in (path/'training'/f'{i}').ls()]
        ).float()/255
        if i == 0: ims = new_ims
        else: ims = torch.cat([ims,new_ims])
    train_ims = ims.view(-1,im_size)
    # lbls
    train_lbls = []
    for i in range(n_cls):
        l = L([0]*n_cls)
        l[i] = 1
        train_lbls += [l] * len((path/'training'/f'{i}').ls())    
    train_lbls = tensor(train_lbls)
    ## Valid
    # ims
    for i in range(n_cls):
        new_ims = torch.stack(
            [tensor(Image.open(fn)) for fn in (path/'testing'/f'{i}').ls()]
        ).float()/255
        if i == 0: ims = new_ims
        else: ims = torch.cat([ims,new_ims])
    valid_ims = ims.view(-1,im_size)
    # lbls
    valid_lbls = []
    for i in range(n_cls):
        l = L([0]*n_cls)
        l[i] = 1
        valid_lbls += [l] * len((path/'testing'/f'{i}').ls())    
    valid_lbls = tensor(valid_lbls)
    ## DataLoaders
    train_ds = L(zip(train_ims, train_lbls))
    valid_ds = L(zip(valid_ims, valid_lbls))
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=True)
    return train_dl

### Model ###
def init_mod(im_size, n_cls, hidden_params):
    mod = nn.Sequential(
        nn.Linear(im_size,hidden_params),
        nn.ReLU(),
        nn.Linear(hidden_params,n_cls)
    )
    return mod

### Create SGD Stepper; args = (mod.parameters(), lr) ###
class ParamStepper:
    def __init__(self, p, lr): self.p,self.lr = list(p),lr # remembers your params & lr
        
    def step(self, *args, **kwargs):                       # take one step in the optimal direction
        for o in self.p: o.data -= o.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):                  # zeros out gradients
        for o in self.p: o.grad = None

### Calculate accuracy over entire dl ###
def validate_epoch(dl, mod):
    a = [acc(mod(xb), yb) for xb,yb in dl]         # Gradients calculated & stored at mod(xb) call
    return round(torch.stack(a).mean().item(), 5)  # Avg over all mini-batches, return scalar (not tensor)

### Adjust parameters w/ stepper for each mini-batch in a dl
def train_once(dl, mod, stepper):
    for xb,yb in dl:
        calc_grad(xb, yb, mod)
        stepper.step()
        stepper.zero_grad()

### Calculate gradients for use in train_once ###
def calc_grad(x,y,mod):
    yp = mod(x)
    ls = loss(yp,y)
    ls.backward()

### Run `train_once` `epochs` times given data `dl`, model `mod`, and stepper `stepper`
def train_model(dl, mod, stepper, epochs):
    l = L()
    for i in range(epochs):
        train_once(dl, mod, stepper)
        # print(validate_epoch(dl, mod), end='\t')
        l += validate_epoch(dl, mod)
    return l

### Perform n training sessions ###
def train_model_n_times(dl, im_size, n_cls, hidden_params, epochs, lr, n):
    o = L()
    print('Current Session:',end='  ')
    for i in range(n):
        print(i,end='  ')
        mod = init_mod(im_size, n_cls, hidden_params)
        stepper = ParamStepper(mod.parameters(), lr)
        o += train_model(dl, mod, stepper, epochs)
    print('Done')
    return tensor(o).reshape(n,epochs)
    
### Loss & Accuracy ###
def softmax(t):
    if len(t.shape) == 1: return torch.exp(t) / torch.exp(t).sum()
    else:                 return torch.exp(t) / torch.exp(t).sum(dim=1, keepdim=True)
def loss(yp, y):
    yps = softmax(yp)
    return (1 - (y * yps).sum(dim=1, keepdim=True)).mean()
def acc(yp,y):
    yp_max,yp_i = torch.max(yp, dim=1, keepdim=True)
    y_max, y_i  = torch.max(y,  dim=1, keepdim=True)
    return (yp_i==y_i).float().mean()

In [ ]:
### Init ###
path          = untar_data(URLs.MNIST)
n_cls         = 10
im_size       = 28*28
batch_size    = 64*2*2*2
dl            = init_data(path, im_size, n_cls, batch_size)

In [ ]:
mod = init_mod(im_size, n_cls, hidden_params)
stepper = ParamStepper(mod.parameters(),lr)
train_model(dl, mod, stepper, epochs)

(#20) [0.33621,0.49281,0.65015,0.78121,0.8036,0.81546,0.8212,0.82522,0.82848,0.8305...]

In [ ]:
### Training ###
test1 = train_model_n_times(dl, im_size, n_cls, hidden_params=30, epochs=50, lr=.1, n=10)

Current Session:  0  1  2  3  4  5  6  7  8  9  Done


In [ ]:
import pandas as pd
df = pd.DataFrame(test1.numpy())
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.34518,0.38481,0.63659,0.65775,0.72319,0.73800,0.74455,0.74843,0.80427,0.81278,...,0.84084,0.84109,0.84129,0.84086,0.84203,0.84222,0.84172,0.84287,0.84270,0.84317
1,0.27709,0.50341,0.59779,0.71376,0.73304,0.74010,0.74500,0.74773,0.75045,0.75266,...,0.92039,0.92127,0.92131,0.09890,0.09868,0.09868,0.09868,0.09875,0.09868,0.09868
2,0.54159,0.48585,0.64357,0.65320,0.65737,0.66076,0.66368,0.66490,0.66585,0.70725,...,0.75614,0.75692,0.75694,0.75679,0.75701,0.75762,0.75804,0.75808,0.75792,0.75898
3,0.24039,0.47394,0.64107,0.65256,0.65881,0.66215,0.66439,0.66647,0.67853,0.72142,...,0.91837,0.91861,0.91999,0.92063,0.92152,0.92169,0.92247,0.92234,0.09890,0.09883
4,0.24443,0.59117,0.65282,0.66003,0.66554,0.66843,0.67730,0.74410,0.74721,0.75038,...,0.76985,0.77029,0.76995,0.77002,0.77106,0.77064,0.77116,0.77111,0.77193,0.77128
5,0.24930,0.47674,0.59392,0.70975,0.72639,0.73461,0.73931,0.74260,0.80062,0.81894,...,0.91788,0.91874,0.09890,0.09868,0.09854,0.09890,0.09847,0.09847,0.09875,0.09868
6,0.51975,0.51805,0.67444,0.72549,0.80854,0.81771,0.82331,0.83686,0.87542,0.88332,...,0.92269,0.92348,0.92337,0.09875,0.09847,0.09904,0.09861,0.09875,0.09890,0.09890
7,0.52406,0.48904,0.62592,0.70262,0.71614,0.72606,0.73039,0.73366,0.73588,0.73756,...,0.83001,0.83048,0.83027,0.83114,0.83071,0.81577,0.09890,0.09854,0.09832,0.09854
8,0.28422,0.54772,0.63704,0.71645,0.76977,0.81235,0.81922,0.82369,0.82690,0.82964,...,0.85110,0.85181,0.85203,0.85225,0.85240,0.85226,0.09832,0.09875,0.09890,0.09868
9,0.25515,0.44452,0.69613,0.72194,0.73582,0.74087,0.74502,0.74759,0.74974,0.75161,...,0.76849,0.76867,0.76893,0.77047,0.80339,0.81678,0.82339,0.82577,0.82960,0.83089


## Performance Tweaks

### Leaky ReLU